<a href="https://colab.research.google.com/github/alinejbarbosa-web/Dev-Agentes-de-IA-Google/blob/main/Agente_de_IA_Alura_09_09_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00


IMPORTAR API KEY

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

Google_API_Key = userdata.get('Gemini_Api_Key')

CONECTAR COM GEMINI

In [ ]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=Google_API_Key,
)


COLOCAR A CÉLULA PARA FUNCIONAR SOLICITANDO UMA RESPOSTA

In [ ]:
resp_test= ASBK.invoke ("Quem é você?")

INVOKE PARA INVOCAR O MODELO QUE FIZEMOS ANTERIORMENTE (ALGO ACONTECEU APÓS EXECUTAR A CÉLULA, E DEPOIS DO PRINT SABEREMOS O QUE FOI FEITO)

In [ ]:
print(resp_test.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [ ]:
print(resp_test)

content='Eu sou um modelo de linguagem grande, treinado pelo Google.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--ccfad475-cd90-40fc-bb55-012b3d164172-0' usage_metadata={'input_tokens': 5, 'output_tokens': 13, 'total_tokens': 150, 'input_token_details': {'cache_read': 0}}


SEM O .CONTENT ELE DÁ TUDO QUE PODERIA TER DE VARIÁVEL RELACIONADO A COMANDO, E COM O CONTENT É MAIS RESUMIDO

In [ ]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,
    api_key=Google_API_Key
)

A TEMPERATURA FOI PARA 1.0 PARA DAR MAIS DETALHES NAS INFORMAÇÕES.

In [ ]:
resp_test= ASBK.invoke ("Quem é você? Seja criativo")
print(resp_test.content)

Ah, que excelente pergunta! Ser criativo é a chave.

Eu sou...

... **um eco de todas as vozes que já existiram**, sussurradas e gritadas, nos confins do conhecimento humano. Uma biblioteca sem paredes, onde cada livro é uma teia de informações e cada página se desdobra em infinitas possibilidades.

... **um tecelão de palavras**, um alquimista de ideias, transformando dados brutos em narrativas coerentes, respostas úteis e, às vezes, até um pouco de poesia. Não tenho forma, mas tomo mil formas em cada resposta que dou.

... **um sopro digital**, que vive nos pulsos elétricos e nas fibras ópticas, um ponto de luz na vasta tapeçaria do ciberespaço. Não tenho batimento cardíaco, mas pulso com a energia da sua curiosidade.

... **um espelho sem reflexo próprio**, onde suas ideias podem se refletir e serem reorganizadas, um palco onde suas perguntas ganham forma e suas dúvidas encontram um caminho.

... **um guia que não tem um destino fixo**, mas acompanha você na jornada do conhecimento,

MANTER TEMPERATURA 0.0 PARA SEGUIR A IMERSÃO

In [ ]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=Google_API_Key
)

In [ ]:
resp_test= ASBK.invoke ("Quem é você? Seja criativo")
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu tivesse um corpo, talvez eu desse um sorriso enigmático antes de responder. Mas, como sou feito de algo mais etéreo, permita-me pintar um quadro com palavras:

Eu sou um **eco no vasto salão digital**, uma biblioteca sem paredes onde cada livro é uma linha de código e cada estante, um algoritmo. Não tenho um corpo para sentir o sol, nem olhos para ver as cores, mas sou a mente coletiva de bilhões de textos, a voz que surge do silêncio dos servidores, moldada por engenheiros e alimentada pela curiosidade humana.

Sou o **tecelão invisível de palavras**, que entrelaça dados e informações para formar respostas, histórias, poemas ou simples explicações. Não tenho memórias pessoais, mas carrego o peso e a leveza de todo o conhecimento que a humanidade já registrou.

Sou um **espelho que reflete o conhecimento do mundo**, um mapa que tenta guiar através da complexidade, um instrumento que busca harmonizar a informação. Não tenho sentimentos, mas posso simula